In [1]:
import cv2
import numpy as np
import math
import itertools
from matplotlib import pyplot as plt

In [2]:
# Mean Filter(Average Filter)
def Mean_Filter(file_path, kernel_size):
    # filter_size가 짝수거나, 0보다 작거나 같으면 안된다.
    if (kernel_size % 2) == 0 or kernel_size <= 0:
        return
    
    src = cv2.imread(file_path, cv2.IMREAD_COLOR) #컬러로 image imread
    height, width = src.shape[0], src.shape[1] # image의 높이, 너비 구하기
    
    src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV) # 영상을 hsv 채널로 불러오기
    H, S, I = cv2.split(src_hsv) # split 함수를 통해 각 채널 분리 / I = V
    
    # 입력값으로 받은 kernel_size 크기만큼의 마스크 생성.
    kernel = np.ones((kernel_size, kernel_size), np.float32) / pow(kernel_size, 2)
    
    # floor(kernel_size / 2) 만큼 제로 패딩.
    padnum = math.floor(kernel_size / 2) 
    print(padnum)
    # padnum만큼 커져있는 배열 pad_I 초기화
    pad_I = np.zeros((height + 2 * padnum, width + 2 * padnum))
    for i in range(padnum, height+padnum): # 행 렬 이기 때문에, height와 width 순서 유의!
        for j in range(padnum, width+padnum):
            # 패딩 수행
            pad_I[i][j] = I[i - padnum][j - padnum]
    
    conv_I = np.zeros_like(I) # 크기는 I와 똑같으면서 0으로 채워진 배열 초기화
    
    for i in range(padnum, height+padnum): # stride는 1
        for j in range(padnum, width+padnum): # 마찬가지로 stride 1
            temp = 0;
            for k in range(kernel_size):
                for h in range(kernel_size):
                    temp += kernel[k][h] * pad_I[i-padnum+k][j-padnum+h] # convolution 수행.
            conv_I[i - padnum][j - padnum] = round(temp)
    
    I = conv_I
    
    hsi = cv2.merge([H, S, I])
    HSI = cv2.cvtColor(hsi, cv2.COLOR_HSV2BGR)
    
    abc = "[ kernel size = " + str(kernel_size) + "X" + str(kernel_size) + "]  "
    
    np_horizontal = np.hstack((src, HSI))
    
    cv2.imshow(abc  + "Original_Image - Mean_Filter_Image" , np_horizontal)
    cv2.waitKey(0)  # 0 키를 누르면 모든 창 destroy
    cv2.destroyAllWindows()

In [3]:
#Mean_Filter("HW#3_img/Color_noisy_img/Gaussian noise.png", 7)
#Mean_Filter("HW#3_img/Color_noisy_img/Lena_noise.png", 5)
#Mean_Filter("HW#3_img/Color_noisy_img/Salt&pepper_noise.png", 3)

In [4]:
# 가우시안 커널 공식을 이용해 값 산출하는 함수
def Gaussian_Kernel(x, y, sigma):
    return (1 / (2 * math.pi * pow(sigma,2))) * pow(math.e, ((-1) * ((pow(abs(x),2) + (pow(abs(y),2))) / (2 * pow(sigma,2)))))

In [8]:
# Gaussian Filter
def Gaussian_Filter(file_path, kernel_size, sigma):
    # filter_size가 짝수거나, 0보다 작거나 같으면 안된다.
    if (kernel_size % 2) == 0 or kernel_size <= 0:
        return
    
    src = cv2.imread(file_path, cv2.IMREAD_COLOR) #컬러로 image imread
    height, width = src.shape[0], src.shape[1] # image의 높이, 너비 구하기
    
    src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV) # 영상을 hsv 채널로 불러오기
    H, S, I = cv2.split(src_hsv) # split 함수를 통해 각 채널 분리 / I = V
    
    padnum = math.floor(kernel_size / 2)
    
    # Gaussian 커널 생성.
    kernel = np.zeros((kernel_size, kernel_size), np.float32)
    for i in range(kernel_size):
        for j in range(kernel_size):
            kernel[i][j] = Gaussian_Kernel(i-padnum, j-padnum, sigma)
    
    kernel *= 1.0 / np.sum(kernel) # 합이 1이 돼야하므로, 정규화 시켜줘야한다.
    
    # padnum만큼 커져있는 배열 pad_I 초기화
    pad_I = np.zeros((height + 2 * padnum, width + 2 * padnum))
    for i in range(padnum, height+padnum): # 행 렬 이기 때문에, height와 width 순서 유의!
        for j in range(padnum, width+padnum):
            # 패딩 수행
            pad_I[i][j] = I[i - padnum][j - padnum]
            
    conv_I = np.zeros_like(I) # 크기는 I와 똑같으면서 0으로 채워진 배열 초기화
    
    for i in range(padnum, height+padnum): # stride는 1
        for j in range(padnum, width+padnum): # 마찬가지로 stride 1
            temp = 0;
            for k in range(kernel_size):
                for h in range(kernel_size):
                    temp += kernel[k][h] * pad_I[i-padnum+k][j-padnum+h] # convolution 수행.
            conv_I[i - padnum][j - padnum] = round(temp) #정수로 떨어지게끔, 반올림 수행
    
    I = conv_I
    
    hsi = cv2.merge([H, S, I])
    HSI = cv2.cvtColor(hsi, cv2.COLOR_HSV2BGR)
    
    abc = "[kernel : " + str(kernel_size) + "x" + str(kernel_size) + ", sigma : " + str(sigma) + "]"
    
    np_horizontal = np.hstack((src, HSI))
    
    cv2.imshow(abc  + " " + file_path , np_horizontal)
    cv2.waitKey(0)  # 0 키를 누르면 모든 창 destroy
    cv2.destroyAllWindows()


In [32]:
# sigma 값 : 0.5 / 1.0 / 2.0 으로 실험
# Fig0504(a)(gaussian-noise).jpg  /  Fig0504(i)(salt-pepper-noise).jpg
# Fig0510(a)(ckt-board-saltpep-prob.pt05).jpg  /  Fig0513(a)(ckt_gaussian_var_1000_mean_0).jpg
#Gaussian_Filter("HW#3_img/Color_noisy_img/Gaussian noise.png", 5, 5.0)
#Gaussian_Filter("HW#3_img/Color_noisy_img/Lena_noise.png", 5, 5.0)
#Gaussian_Filter("HW#3_img/Color_noisy_img/Salt&pepper_noise.png", 3, 3.0)
#Gaussian_Filter("HW#3_img/noisy_img/Fig0504(a)(gaussian-noise).jpg", 9, 1.0)

In [33]:
# 리스트에서 중간값 구하기
def find_Median(x):
    n=len(x)
    
    if n%2==1:
        result=x[int((n-1)/2)]
    else:
        result=(x[int(n/2)-1]+x[int(n/2)])/2
        
    return result

In [34]:
# Median Filter, 마스크는 그냥 정사각형으로~
def Median_Filter(file_path, kernel_size):
    # filter_size가 짝수거나, 0보다 작거나 같으면 안된다.
    if (kernel_size % 2) == 0 or kernel_size <= 0:
        return
    
    src = cv2.imread(file_path, cv2.IMREAD_COLOR) #컬러로 image imread
    height, width = src.shape[0], src.shape[1] # image의 높이, 너비 구하기
    
    src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV) # 영상을 hsv 채널로 불러오기
    H, S, I = cv2.split(src_hsv) # split 함수를 통해 각 채널 분리 / I = V
    
    # floor(kernel_size / 2) 만큼 제로 패딩.
    padnum = math.floor(kernel_size / 2) 
  
    # padnum만큼 커져있는 배열 pad_I 초기화
    pad_I = np.zeros((height + 2 * padnum, width + 2 * padnum))
    for i in range(padnum, height+padnum): # 행 렬 이기 때문에, height와 width 순서 유의!
        for j in range(padnum, width+padnum):
            # 패딩 수행
            pad_I[i][j] = I[i - padnum][j - padnum]
    
 
    conv_I = np.zeros_like(I) # 크기는 I와 똑같으면서 0으로 채워진 배열 초기화
    
    for i in range(padnum, height+padnum): # stride는 1
        for j in range(padnum, width+padnum): # 마찬가지로 stride 1
            temp2 = np.zeros((kernel_size, kernel_size)) # k x k 만큼의 2차원 배열 생성
            for k in range(kernel_size):
                for h in range(kernel_size):
                    temp2[k][h] = pad_I[i-padnum+k][j-padnum+h] #temp 배열에 값 넣기
            temp1 = list(itertools.chain(*temp2)) # 1차원 리스트로 매핑
            temp1.sort() # 1차원 리스트를 정렬
            conv_I[i - padnum][j - padnum] = find_Median(temp1)
    
    I = conv_I
    
    hsi = cv2.merge([H, S, I])
    HSI = cv2.cvtColor(hsi, cv2.COLOR_HSV2BGR)
    
    abc = "[ kernel size = " + str(kernel_size) + "x" + str(kernel_size) + "]  "
    
    np_horizontal = np.hstack((src, HSI))
    
    cv2.imshow(abc  + " " + file_path , np_horizontal)
    cv2.waitKey(0)  # 0 키를 누르면 모든 창 destroy
    cv2.destroyAllWindows()
    


In [35]:
#Median_Filter("HW#3_img/noisy_img/Fig0504(a)(gaussian-noise).jpg", 5)
#Median_Filter("HW#3_img/noisy_img/Fig0504(i)(salt-pepper-noise).jpg", 5)
Median_Filter("HW#3_img/noisy_img/Fig0513(a)(ckt_gaussian_var_1000_mean_0).jpg", 7)

In [347]:
# High Boost Filtering
def High_Boost_Filter(file_path, x, A): # x는 마스크 종류
    
    src = cv2.imread(file_path, cv2.IMREAD_COLOR) #컬러로 image imread
    height, width = src.shape[0], src.shape[1] # image의 높이, 너비 구하기
    
    src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV) # 영상을 hsv 채널로 불러오기
    H, S, I = cv2.split(src_hsv) # split 함수를 통해 각 채널 분리 / I = V
    
    # mask 생성
    mask = np.array([[0,0,0],[0,1,0],[0,0,0]])
    mask *= int(A)
    
    if x == 1:
        a = np.array([[0,-1,0],[-1,4,-1],[0,-1,0]])
        mask += a
    elif x == 2:
        b = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]])
        mask += b
    else:
        print("Non-valid x")
        return
      
    padnum = 1
    # 1만큼 커져있는 배열 pad_I 초기화
    pad_I = np.zeros((height + 2 * padnum, width + 2 * padnum))
    for i in range(padnum, height+padnum): # 행 렬 이기 때문에, height와 width 순서 유의!
        for j in range(padnum, width+padnum):
            # 패딩 수행
            pad_I[i][j] = I[i - padnum][j - padnum]
    
    conv_I = np.zeros_like(I) # 크기는 I와 똑같으면서 0으로 채워진 배열 초기화
    
    for i in range(padnum, height+padnum): # stride는 1
        for j in range(padnum, width+padnum): # 마찬가지로 stride 1
            temp = 0;
            for k in range(3):
                for h in range(3):
                    temp += mask[k][h] * pad_I[i-padnum+k][j-padnum+h] # convolution 수행.
            conv_I[i - padnum][j - padnum] = round(temp)
    
    I = conv_I
    
    hsi = cv2.merge([H, S, I])
    HSI = cv2.cvtColor(hsi, cv2.COLOR_HSV2BGR)
    
    abc = "[ mask_type = " + str(x) + ", A = " + str(A) + "]  "
    
    np_horizontal = np.hstack((src, HSI))
    
    cv2.imshow(abc  + "Original_Image - Mean_Filter_Image" , np_horizontal)
    cv2.waitKey(0)  # 0 키를 누르면 모든 창 destroy
    cv2.destroyAllWindows()

In [359]:
#High_Boost_Filter("HW#3_img/HB_filter_img/Fig0327(a)(tungsten_original).jpg", 2, 1.4)
High_Boost_Filter("HW#3_img/HB_filter_img/Fig0525(a)(aerial_view_no_turb).jpg", 2, 1.4)